# ResNet - SSD

In [8]:
# Model list
model_list = ["../Model/CNN_model_V1.h5", 
              "../Model/CNN_model_V2.h5"]

In [9]:
import numpy as np
import tensorflow as tf
import time
from tensorflow.keras.models import load_model
from model_architecture import get_model
import imutils
import cv2

model = get_model()
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 48, 48, 1)]       0         
                                                                 
 conv1_1 (Conv2D)            (None, 48, 48, 64)        640       
                                                                 
 batch_normalization_34 (Bat  (None, 48, 48, 64)       256       
 chNormalization)                                                
                                                                 
 conv1_2 (Conv2D)            (None, 48, 48, 64)        36928     
                                                                 
 batch_normalization_35 (Bat  (None, 48, 48, 64)       256       
 chNormalization)                                                
                                                                 
 pool1_1 (MaxPooling2D)      (None, 24, 24, 64)        0   

In [30]:
print("[INFO] Loading CNN model...")
labels = ["Angry", "Happy", "Neutral", "Sad", "Surprise"]
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.load_weights(model_list[-1])

print("[INFO] Loading ResNetSSD model ...")
prototxt_file = "../utils/ResNet_SSD/deploy.prototxt"
caffemodel_file = "../utils/ResNet_SSD/Res10_300x300_SSD_iter_140000.caffemodel"
detector = cv2.dnn.readNetFromCaffe(prototxt_file, caffemodel_file)

print("[INFO] Loading camera/video ...")
camera = cv2.VideoCapture(0)

while True:
    ret, image = camera.read()
    
    if isinstance(image,type(None)): break
    
    start = time.time()
    
    base_img = image.copy()
    original_size = base_img.shape
    target_size = (700, 400)
    image = cv2.resize(image, target_size)
    aspect_ratio_x = (original_size[1] / target_size[1])
    aspect_ratio_y = (original_size[0] / target_size[0])
            
    origin_h, origin_w = image.shape[:2]

    img_blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    
    detector.setInput(img_blob)
    
    detections = detector.forward()

    try:
        for i in range(0, detections.shape[2]):
        
            confidence = detections[0, 0, i, 2]

            if confidence > 0.5:

                bounding_box = detections[0, 0, i, 3:7] * np.array([origin_w, origin_h, origin_w, origin_h])

                x_start, y_start, x_end, y_end = bounding_box.astype('int')

                face = image[int(y_start):int(y_end), int(x_start):int(x_end)]
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY) #transform to gray scale
                face = cv2.resize(face, (48, 48))

                img_pixels = np.array(face)
                img_pixels = np.expand_dims(img_pixels, axis = 0)

                img_pixels = img_pixels / 255.0

                predictions = model.predict(img_pixels)

                pred = predictions.argmax()

                score = int(np.amax(predictions) * 100)

                emotion = labels[pred]

                # bounding box
                cv2.rectangle(image, (x_start, y_start), (x_end, y_end),(0, 0, 255), 2)
                cv2.rectangle(image, (x_start, y_start-18), (x_end, y_start), (0, 0, 255), -1)
                cv2.putText(image, emotion + ": %.0f" % score + "%"
                            , (x_start+2, y_start-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    except Exception as ex:
        print(ex)

    # Calc FPS
    end = time.time()
    totalTime = end - start
    if totalTime == 0:
        totalTime = 1
    
    fps = 1 // totalTime
    cv2.putText(image, "FPS: " + str(fps), (15, int(origin_h * 0.1)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.imshow('Facial Expression', image)
    
    if cv2.waitKey(5) != -1:
        break

camera.release()
cv2.destroyAllWindows()

[INFO] Loading CNN model...
[INFO] Loading ResNetSSD model ...
[INFO] Loading camera/video ...


# Haar Cascade

In [ ]:
import numpy as np
import cv2
import tensorflow as tf
import time
from model_architecture import get_model

print("[INFO] Loading CNN model...")
labels = ["Angry" ,"Happy", "Neutral", "Sad", "Surprise"]
model = get_model()
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.load_weights(model_list[-1])

print("[INFO] Loading HaarCascade model...")
face_detection = cv2.CascadeClassifier("../utils/Haar_Cascade/haar_cascade_face_detection.xml")

print("[INFO] Loading Camera/Video...")
camera = cv2.VideoCapture(0)

settings = {
    'scaleFactor': 1.3,
    'minNeighbors': 5,
    'minSize': (50, 50)
}

while True:
    ret, img = camera.read()
    
    origin_h, origin_w = img.shape[:2]
    
    start = time.time()
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    detected = face_detection.detectMultiScale(gray, **settings)

    try:
        for (x, y, w, h) in detected:
        
            cv2.rectangle(img, (x, y), (x + w, y + h), (245, 135, 66), 2)
            cv2.rectangle(img, (x, y-18), (x + w, y), (245, 135, 66), -1)

            face = gray[int(y):(int(y) + int(h)), int(x):(int(x) + int(w))]
            face = cv2.resize(face, (48, 48))

            img_pixels = np.array(face)
            img_pixels = np.expand_dims(img_pixels, axis = 0)

            img_pixels = img_pixels / 255.0

            predictions = model.predict(img_pixels)

            pred = predictions.argmax()

            score = int(np.amax(predictions) * 100)

            state = labels[pred]

            cv2.putText(img, state + ": %.0f" % score + "%",
                        (x + 2, y -5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

    except Exception as ex:
        print(ex)
        
    # Calc FPS
    end = time.time()
    totalTime = end - start
    if totalTime == 0:
        totalTime = 1
    fps = 1 // totalTime
    
    cv2.putText(img, "FPS: " + str(fps), (15, int(origin_h * 0.1)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.imshow('Facial Expression', img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # writer.writeFrame(img)
    if cv2.waitKey(5) != -1:
        break

camera.release()
cv2.destroyAllWindows()

[INFO] Loading CNN model...
[INFO] Loading HaarCascade model...
[INFO] Loading Camera/Video...


# 7 Classes FER Demo

In [10]:
import numpy as np
import cv2
import tensorflow as tf
import time
from tensorflow.keras.models import load_model

print("[INFO] Loading CNN model...")
labels = ["Angry", "Disgust", "Fear","Happy", "Neutral", "Sad", "Surprise"]
model = load_model(model_list[0])

print("[INFO] Loading HaarCascade model...")
face_detection = cv2.CascadeClassifier("../utils/Haar_Cascade/haar_cascade_face_detection.xml")

print("[INFO] Loading Camera/Video...")
camera = cv2.VideoCapture(0)

settings = {
    'scaleFactor': 1.3,
    'minNeighbors': 5,
    'minSize': (50, 50)
}

while True:
    ret, img = camera.read()
    
    origin_h, origin_w = img.shape[:2]
    
    start = time.time()
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    detected = face_detection.detectMultiScale(gray, **settings)

    try:
        for (x, y, w, h) in detected:
        
            cv2.rectangle(img, (x, y), (x + w, y + h), (245, 135, 66), 2)
            cv2.rectangle(img, (x, y-18), (x + w, y), (245, 135, 66), -1)

            face = gray[int(y):(int(y) + int(h)), int(x):(int(x) + int(w))]
            face = cv2.resize(face, (48, 48))

            img_pixels = np.array(face)
            img_pixels = np.expand_dims(img_pixels, axis = 0)

            img_pixels = img_pixels / 255.0

            predictions = model.predict(img_pixels)

            pred = predictions.argmax()

            score = int(np.amax(predictions) * 100)

            state = labels[pred]

            cv2.putText(img, state + ": %.0f" % score + "%",
                        (x + 2, y -5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

    except Exception as ex:
        print(ex)
        
    # Calc FPS
    end = time.time()
    totalTime = end - start
    if totalTime == 0:
        totalTime = 1
    fps = 1 // totalTime
    
    cv2.putText(img, "FPS: " + str(fps), (15, int(origin_h * 0.1)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.imshow('Facial Expression', img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # writer.writeFrame(img)
    if cv2.waitKey(5) != -1:
        break

camera.release()
cv2.destroyAllWindows()

[INFO] Loading CNN model...
[INFO] Loading HaarCascade model...
[INFO] Loading Camera/Video...
